In [1]:
# Installing libraries
!pip install tensorflow
# Imports
from pathlib import Path
import tensorflow as tf
import datetime
import json
import os

# SUS Pills detection model

## Explanation of the imports:

| Import name | Usage |
| - | - |
| pathlib.Path | folder/directory management |
| tensorflow | creating AI model |
| datetime | creating model name |
| json | unpacking parameters from file |
| os | rising Tensorflow min log level |

## Clearing unnecessary warnings caused by Tensorflow:

In [2]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

## Constant variables:

In [3]:
# Model save thresholds
MAX_LOSS = 0.5
MIN_ACC = 0.9

# Verbose level
VERBOSE = 2

# Directories
DATA_DIR = "..\\data\\images\\.train"
SAVE_DIR = ".\\models"
LOG_DIR = ".\\.logs"

# Files
PARAMETERS_F = ".\\parameters\\parameters_list.json"

### create_model function:

Create a package (model, test dataset, train dataset, batch_size, learning rate, epochs). Use the predefined dictionary with desired parameters. Every dictionary should look like that:

```
{
    "input_y": <integer>,
    "input_x": <integer>,
    "batch_size": <integer>,
    "dimensions": <integer>,
    "conv_filters": <list of integers>,
    "dense_units": <list of integers>,
    "drop_prob": <list of floats>,
    "learning_rate": <float>,
    "epochs": <integer>
}
```

This script already uses a pre-existing ```*.json``` file with a list of directories used by the ```map()``` function by the end of this script. You can edit it to create different models. Core features of models are static.

In [4]:
def create_package(params: dict) -> dict:
    """
    Load dataset and create the model based on given parameters.
    """


    """ SET VARIABLES """
    # Number of pill types we want the model to be albe to predict
    outputs = len(list(Path(DATA_DIR).iterdir()))

    # Inputs: y = height, x = width
    input_y = params['input_y']
    input_x = params['input_x']

    # Batch size, learning rate, and epochs
    batch_size = params['batch_size']
    learning_rate = params['learning_rate']
    epochs = params['epochs']

    # Dimensions and color mode
    dimensions = params['dimensions']
    if dimensions == 1:
        color_mode = 'grayscale'
    elif dimensions == 3:
        color_mode = 'rgb'
    
    # Convolution filters, dense units, and dropout probability
    filters_list = params['conv_filters']
    dense_block = zip(params['dense_units'], params['drop_prob'])

    # Regularizer used for convolutions
    regul_l2 = tf.keras.regularizers.L2()


    """ LOAD DATASET """
    ds_train, ds_test = tf.keras.utils.image_dataset_from_directory(
        directory=DATA_DIR,
        label_mode="int",
        batch_size=batch_size,
        image_size=(input_y, input_x),
        validation_split=0.2,
        seed=123,
        subset="both",
        color_mode=color_mode,
    )


    """ CREATE MODEL """
    model = tf.keras.Sequential()

    # Add input layer
    model.add(tf.keras.layers.Input((input_y, input_x, dimensions)))

    # Add conv2d, maxpool, and batchnorm layers
    for filters in filters_list:
        model.add(tf.keras.layers.Conv2D(filters, kernel_size=3,
                  padding='same', activation='relu', kernel_regularizer=regul_l2))
        model.add(tf.keras.layers.MaxPool2D(pool_size=5, padding='same'))
        model.add(tf.keras.layers.BatchNormalization())

    # Add flatten layer
    model.add(tf.keras.layers.Flatten())

    # Add dense and dropout layers
    for units, drop_prob in dense_block:
        model.add(tf.keras.layers.Dense(units, activation='relu'))
        model.add(tf.keras.layers.Dropout(rate=drop_prob))

    # Add output layer
    model.add(tf.keras.layers.Dense(outputs, activation='relu'))


    """ CREATE A PACKAGE """
    package = {
        'model': model,
        'train': ds_train,
        'test': ds_test,
        'batch_size': batch_size,
        'learning_rate': learning_rate,
        'epochs': epochs,
    }

    return package

## Load the parameters from the file and map all of its contents to create packages:

In [5]:
with open(PARAMETERS_F, 'r') as f:
    parameters_list = json.load(f)

In [6]:
packages = list(map(create_package, parameters_list))

Found 796 files belonging to 15 classes.
Using 637 files for training.
Using 159 files for validation.


## Train, evaluate, and save the model extracted from every package:

In [7]:
for package in packages:

    # Unpack all variables
    model = package['model']
    train = package['train']
    test = package['test']
    batch_size = package['batch_size']
    learning_rate = package['learning_rate']
    epochs = package['epochs']

    # Print the summary
    if (VERBOSE == 2):
        model.summary()

    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=[tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=True)],
        metrics=["accuracy"],
    )

    # Create the model's name
    model_name = f"cnn-{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"

    # logging directory
    log_dir = f'{LOG_DIR}\\{model_name}'

    # Initiate TensorBoard
    tensorboard = tf.keras.callbacks.TensorBoard(
        log_dir=log_dir, histogram_freq=1)

    # Fit the dataset to the model
    model.fit(train, batch_size=batch_size, epochs=epochs,
              verbose=VERBOSE, callbacks=[tensorboard])

    # Evaluate the model
    evaluate = model.evaluate(test, batch_size=batch_size,
                              verbose=VERBOSE)

    # Save the model if plausible
    eval_loss = evaluate[0]
    eval_accuracy = evaluate[1]
    if (eval_loss < MAX_LOSS) and (eval_accuracy > MIN_ACC):
        model.save(f"{SAVE_DIR}\\{model_name}")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 50)        1400      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 50)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 13, 13, 50)       200       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 100)       45100     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 3, 3, 100)        0         
 2D)                                                             
                                                        